In [1]:
!pip3 install websockets pandas ta python-binance python-dotenv

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [2]:
import websockets
import json
import pandas as pd
import asyncio
import ta
import os
from binance.client import Client
from binance.exceptions import BinanceAPIException

from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.


api_key = os.environ['api_key']
api_secret = os.environ['api_secret']
roc_num_records = int(os.environ['roc_num_records'])
#symbol = os.environ['symbol']
#quantity = os.environ['quantity']
quantity = "1"
#symbol = "SHIBUSDT"
symbol = "LINKUSDT"
trailing_coefficient = float(os.environ['trailing_coefficient'])
#rise_percentage = float(os.environ['rise_percentage'])
rise_percentage = 1.005

# Variables
client = Client(api_key, api_secret)
df = pd.DataFrame()
open_position = False
dfOrders = pd.DataFrame()
profit_list = []
totalProfit = 0

# TODO check quantity >= minQty, if not sto execution
symbolinfo = client.get_symbol_info(symbol)
print(f"{symbol}, {quantity}")

#print(f"LOT_SIZE (min qty): {symbol} is {symbolinfo['filters'][2]['minQty']}")
print(f"LOT_SIZE (min qty): {symbolinfo['filters'][2]['minQty']}")
print(f"MIN_NOTIONAL      : {symbolinfo['filters'][3]['minNotional']}")



LINKUSDT, 1
LOT_SIZE (min qty): 0.01000000
MIN_NOTIONAL      : 10.00000000


In [3]:
# Functions
def create_frame(msg):
  df = pd.DataFrame([msg])
  df = df.loc[:,['s', 'E', 'c']]
  df.columns = ['symbol', 'Time', 'Price']
  df.Price = df.Price.astype('float')
  df.Time = pd.to_datetime(df.Time, unit='ms')
  return df

def create_order(symbol, side, type, quantity):
    try:
        order = client.create_order(symbol=symbol, side=side, type=type, quantity=quantity)
    except BinanceAPIException as e:
        print(f"an error occurred while creating {side} order")
        print(e)
    else:
        #print("order created succesfully")
        return order

def create_buy_order(symbol, quantity):
    return create_order(symbol, side=Client.SIDE_BUY, type=Client.ORDER_TYPE_MARKET, quantity=quantity)

def create_sell_order(symbol, quantity):
    return create_order(symbol, side=Client.SIDE_SELL, type=Client.ORDER_TYPE_MARKET, quantity=quantity)

In [4]:
#PROD
stream = websockets.connect(f"wss://stream.binance.com:9443/stream?streams={symbol.lower()}@miniTicker")
#TEST
#stream = websockets.connect('wss://testnet.binance.vision/stream?streams=adausdt@miniTicker')

# TODO make quantity, symbol, roc test rows umber, trailing stop coefficient dynamic
async with stream as receiver: 
  print(f"Starting trailing strategy for symbol {quantity} of {symbol}")
  while True:
    data = await receiver.recv()
    data = json.loads(data)['data']
    df = df.append(create_frame(data))
    
    if len(df) > roc_num_records:
      # If not open position then calculate ROC with roc_num_records records
      if not open_position:
        # TODO understand second condition... place buy order, 
        # save buyprice for later use        
        roc = ta.momentum.roc(df.Price, roc_num_records).iloc[-1]
        #print(f"The actual ROC is {roc}")
        if roc > 0 and ta.momentum.roc(df.Price, roc_num_records).iloc[-2]:
          #print(f"ROC = {roc}")
          order = create_buy_order(symbol=symbol, quantity=quantity)
          #dfOrders.append(pd.DataFrame([order,'BUY']))
          #print(order)
          open_position = True
          buyprice = float(order['fills'][0]['price'])
          print(f"===========================")
          print(f"BUY order created succesfully Qty: {order['fills'][0]['qty']}, Price {buyprice}")

      if open_position:
        subdf = df[df.Time >= pd.to_datetime(
            order['transactTime'], unit='ms')]
        if len(subdf) > 1:

          #Use cummax function to calculate highest price
          subdf['highest'] = subdf.Price.cummax()

          # Setting trailing stop
          # TODO make trailing coefficient variable
          subdf['trailingstop'] = subdf['highest'] * trailing_coefficient

          # If any of these 2 condition is true i want to place a SELL ORDER
          # If the asset is rising 0.02%
          if subdf.iloc[-1].Price < subdf.iloc[-1].trailingstop or df.iloc[-1].Price / float(order['fills'][0]['price']) > rise_percentage:
            order = create_sell_order(symbol=symbol, quantity=quantity)
            print(f"SELL order created succesfully Qty: {order['fills'][0]['qty']}, Price {order['fills'][0]['price']}")
            open_position = False
            #dfOrders.append(pd.DataFrame([order,'SELL']))
            #print(order)

            sellprice = float(order['fills'][0]['price'])
            profit = (sellprice - buyprice)/2
            totalProfit += profit
            profit_list.append(profit)
            #print(f"You made {profit} profit")
            print(f"Transaction profit: {profit}")
            print(f"Total profit      : {totalProfit}\nN. of transactions: {len(profit_list)}")
            print(f"===========================\n\n")

    #print(df.iloc[-1])


Starting trailing strategy for symbol 1 of LINKUSDT
BUY order created succesfully Qty: 1.00000000, Price 32.25


/var/folders/c9/00jg9zn11flgr2plp6lzlpgw0000gn/T/ipykernel_13999/811434082.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subdf['highest'] = subdf.Price.cummax()
/var/folders/c9/00jg9zn11flgr2plp6lzlpgw0000gn/T/ipykernel_13999/811434082.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subdf['trailingstop'] = subdf['highest'] * trailing_coefficient


SELL order created succesfully Qty: 1.00000000, Price 32.41000000
Transaction profit: 0.0799999999999983
Total profit      : 0.0799999999999983
N. of transactions: 1


BUY order created succesfully Qty: 1.00000000, Price 32.42
SELL order created succesfully Qty: 1.00000000, Price 32.25000000
Transaction profit: -0.08500000000000085
Total profit      : -0.005000000000002558
N. of transactions: 2


BUY order created succesfully Qty: 1.00000000, Price 32.29
SELL order created succesfully Qty: 1.00000000, Price 32.13000000
Transaction profit: -0.0799999999999983
Total profit      : -0.08500000000000085
N. of transactions: 3


BUY order created succesfully Qty: 1.00000000, Price 32.15
SELL order created succesfully Qty: 1.00000000, Price 32.08000000
Transaction profit: -0.03500000000000014
Total profit      : -0.120000000000001
N. of transactions: 4


BUY order created succesfully Qty: 1.00000000, Price 32.04
